###### **1º Passo: Importação das bibliotecas Ministério do Desenvolvimento, Indústria, Comércio e Serviços [MDIC]**

In [ ]:
import pandas as pd
from io import BytesIO
import os
import requests
import urllib3

StatementMeta(, d741d1ba-a3aa-453d-a92d-cd5b0517cb71, 3, Finished, Available, Finished)

###### **2º Passo: Definção do parametro de ínicio e fim da extração e local de salvar os dados**

In [ ]:
# Parametros para extração dos dados e local destino de armazenamento
data_inicio = 1997
data_fim    = 2025

# Local de salvar os dados lakehouse
raw_dados_publicos_abfss_files = "abfss://3ed9004c-a27b-47fb-8a19-3a0a8508fbf3@onelake.dfs.fabric.microsoft.com/34b49a7f-cf79-410e-8c7a-2b00f7d44771/Files"

StatementMeta(, d741d1ba-a3aa-453d-a92d-cd5b0517cb71, 4, Finished, Available, Finished)

###### **3º Passo: Configurações gerais**

In [ ]:
pasta_armazenamento = "Mdic_Fatos"

mapa_pastas = {
    "IMP": "Importacao",
    "EXP": "Exportacao"
}

tipos_operacao = ["IMP", "EXP"]

caminho_raiz_local = "/lakehouse/default/Files"
base_url = "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm"

# Monta os caminhos finais
dir_destino_local = f"{caminho_raiz_local}/{pasta_armazenamento}"
dir_destino_abfss = f"{raw_dados_publicos_abfss_files}/{pasta_armazenamento}"

print(f"📂 Salvando fisicamente em: {dir_destino_local}")

StatementMeta(, d741d1ba-a3aa-453d-a92d-cd5b0517cb71, 5, Finished, Available, Finished)

📂 Salvando fisicamente em: /lakehouse/default/Files/Mdic_Fatos


###### **4º Passo: Extração dos dados**

In [ ]:
# LOOP duplo (ano + tipo de dados)

for ano in range(data_inicio, data_fim + 1):
    for tipo in tipos_operacao:
        
        dir_base = f"{caminho_raiz_local}/{pasta_armazenamento}"
        subpasta = mapa_pastas[tipo]
        dir_destino_local = f"{dir_base}/{subpasta}"
        
        url = f"{base_url}/{tipo}_{ano}.csv"
        arquivo_parquet = f"{dir_destino_local}/{tipo}_{ano}.parquet"
        
        print(f"⬇️ Processando: {tipo} - {ano}...")
        
        try:
            response = requests.get(url, timeout=120, verify=False)

            if response.status_code == 200:
                
                os.makedirs(dir_destino_local, exist_ok=True)

                df_temp = pd.read_csv(
                    BytesIO(response.content),
                    sep=";",
                    encoding="latin1",
                    low_memory=False
                )

                df_temp["Tipo_Operacao"] = tipo

                df_temp.to_parquet(arquivo_parquet, index=False)

                del df_temp

                print(f"✅ Sucesso: {subpasta}/{tipo}_{ano}.parquet")

            else:
                print(f"❌ Erro HTTP {tipo} {ano}: {response.status_code}")

        except Exception as e:
            print(f"❌ Erro ao processar {tipo} {ano}: {e}")

print("\n--- Dados organizados em importação e exportação ---")

StatementMeta(, d741d1ba-a3aa-453d-a92d-cd5b0517cb71, 6, Finished, Available, Finished)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'balanca.economia.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'balanca.economia.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'balanca.economia.gov.br'. Adding certificate verification is strongly advised. See: https://urllib

✅ Sucesso: Exportacao/EXP_2025.parquet

--- Dados organizados em importação e exportação ---
